In [1]:
from IPython.display import Image
from IPython.display import HTML, IFrame

<div align='center'  class="alert alert-warning" role="alert">
    <h1 >Финальный проект</h1>
    <h2>Евгении Хомяковой</h2>
    <h4>Специализация 'Машинное обучение и анализ данных'</h4>
    <h1>Прогнозирование числа поездок из зон такси города Нью-Йорка</h1>
    <h6>июнь 2020 года</h6>
</div>

Пожалуйста, прежде чем смотреть дальше, убедитесь, что папка `exp` находится в том же месте, где этот html-файл (иначе картинок не будет)

<div align='left'>Нью-Йоркская комиссия по такси и лимузинам (TLC) предоставляет подробные анонимизированные данные о поездках клиентов с 2009 года. Эти данные о поездках желтого такси, сгруппированные по месяцам каждого года, скачиваю отсюда
</div>
<div align='left'><a href="URL">https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page</a>
</div>
<br>
<div align='left'><strong>Дисклеймер:</strong> из-за ограниченных вычислительных мощностей буду анализировать данные за 2016-2018 годы.
</div>
<div align='left'>
    <strong>Задача проекта:</strong> научиться предсказывать количество поездок в ближайшие часы в каждой зоне такси Нью-Йорка.
</div>
<br>
<div align='left'><strong><em>Задачу прогнозирования таких рядов буду решать с помощью авторегрессионных моделей, прогнозируя каждый ряд независимо.</em></strong>
</div>

###  Зоны такси
С сайта TLC скачиваем shapefile c координатами зон, на которые поделен город:
https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip

Он содержит информацию о 263 зонах, с коррдинатами зон, названиями и ID:

Вот как эти зоны выглядят на карте:

In [2]:
IFrame('exp/zones_gdf.png', 600, 600, unconfined=True)

И в интерактивном режиме:

In [3]:
IFrame('exp/interactive_taxi_zones.html', 800, 600, unconfined=True)

#### Обработка сырых данных:

Сырые данные были отфильрованы от ошибок и аномалий, а затем размечены по этим зонам и агрегированы - получились временные ряды для каждой зоны отдельно.

#### Обучение и тест

Нужно обозначить дату, разделяющую данные на две части - обучение и тест. Отбирать ряды, кластеризовать их, а так же обучать прогнозирующие модели будем на данных до 2018/06/30 23:00, а **предсказывать будем на июль 2018 года**.


Вот такие получаются ряды:

In [4]:
IFrame("exp/interactive_some_timerows.html", 900, 650, unconfined=True)

###  Отбор рядов и визуализация спроса на такси на карте
~~Не все ряды одинаково полезны~~ Не все зоны одинаково востребованы -- прогнозировать спрос имеет смысл в популярных зонах, чтобы понимать, сколько в зоне должно быть доступных машин в каждый час.

#### Посчитаем среднее число поездок из каждой зоны в час и раскрасим регионы в соответствии с полученными значениями.
Работать будем с зонами, из которых среднее число поездок в час не меньше 5, таких зон получется 83 штуки:

In [5]:
IFrame("exp/mean_colored_zones.html", 650, 650, unconfined=True)

### Построение модели
В качестве прогнозирующей модели использовался **XGBoost**. Модель строилась отдельно для каждой из шести задач прогнозирования: $$\hat{y}_{T+i|T}, i=1,\dots,6$$

#### Построение признаковых описаний рядов
Признаковое описание каждого ряда для построения модели формировалось из:
1. Значений времени ряда
2. Самих значений ряда (авторегрессионные признаки)
3. Статистической информации о регионе из сырых данных (эти признаки напрямую не связаны со значениями целевой переменной)

#### Отбор признаков для XGBoost
Некоторые признаки могут быть несущественными или незначимыми для целевой переменной, и их включение в модель приводит к увеличению сложности модели, которую сложнее интерпретировать, а так же больше количество признаков увеличивает время обучения модели и уменьшает надежность предсказаний.

Отбор признаков проводился следующим образом. Для каждой модели, обученной на всех созданных признаках, по **кросс-валидации для временных рядов** оценивалось качество по метрике **r2_score**, а затем признаки фильровались по значению **features_importances** (*XGBR.feature_importances_*) с порогом 0.01.

Для прогнозирующих модели затем подбирались гиперпараметры: модели, обученные на отобранных признаках и с разными гиперпараметрами (через Bayesian Optimization Process) снова оценивались по метрике R2 на кросс-валидации .

Отбор признаков и оптимизация гиперпараметром проводились не для каждого "популярного" ряда. Сначала проводилась кластеризация методом **KMeans**, и уже для кластерных центров проводились эти процессы. Все ряды делились на 3 кластера:

In [6]:
IFrame("exp/clusterization.html", 850, 650, unconfined=True)

Вот так выглядят ряды кластерных центров:

In [7]:
IFrame("exp/centroid_timerows.html", 850, 650, unconfined=True)

### Прогнозирование на июль 2018 года

Выбранными моделями построим для каждой географической зоны и каждого конца истории от 2018.06.30 23:00 до 2018.07.31 17:00 прогнозы на 6 часов вперёд; ошибку прогноза считаем по следующему функционалу:

$$Q_{july2018}=\frac{1}{R∗739∗6}\sum_{r=1}^{R}\sum_{T=2018.06.30 23:00}^{2018.07.31 17:00}\sum_{i=1}^{6}∣\widehat{y}^{r}_{T|T+i}−{y}^{r}_{T+i}| $$ ,
где R - это число рядов (83 у нас), 739 - количество часов в прогнозируемом периоде.

$Q_{july2018}$ составила 15.43.

### Визуализация прогнозов на июль 2018 года
Далее представлена визуализация фактического и прогнозируемого спроса на такси (на период июля 2018 года) в самых популярных зонах такси в виде временных рядов и на карте.

In [8]:
IFrame('exp/real_and_predicted_timerows.gif', 1000, 700, unconfined=True)

In [9]:
IFrame('exp/map_with_predictions.gif', 1000, 650, unconfined=True)

jupyter nbconvert --to slides NYC_TAXI_project_04_presentation.ipynb --SlidesExporter.reveal_theme=serif --SlidesExporter.reveal_scroll=True